# A brief summary of *GDB* and *OpenOCD*
<br>
<div style="opacity: 0.8; font-family: Consolas, Monaco, Lucida Console, Liberation Mono, DejaVu Sans Mono, Bitstream Vera Sans Mono, Courier New; font-size: 12px; font-style: italic;">
    ────────
    for more from the author, visit
    <a href="https://github.com/hazemanwer2000">github.com/hazemanwer2000</a>.
    ────────
</div>

## Table of Contents
* [Commands](#commands)
    * [Establishing a connection](#establishing-a-connection)
    * [Reading from memory](#reading-from-memory)
    * [Writing to memory](#writing-to-memory)
    * [Managing breakpoints](#managing-breakpoints)
    * [Controlling execution flow](#controlling-execution-flow)

<hr>

The *GNU Project Debugger (GDB)* is one of the most commonly used for debugging C/C++ applications.

It allows remote debugging, where the executable file runs on a different machine, which is a requirement when cross-compiling (e.g: cross-compiling with `arm-none-eabi-gcc` for an embedded ARM-based target, on a x86 machine).

In embedded systems, a *GDB server* (e.g: *OpenOCD*) detects and communicates with a target, receiving commands from a *GDB* process.

### Commands

#### Establishing a connection

To initiate a GDB debugging session,

In [ ]:
$ gdb-multiarch                          # Supports ARM targets, among others

To allow GDB to read and parse the executable file,

In [ ]:
(gdb) file FILE

To connect to a remote OpenOCD server from within the GDB process, 

In [ ]:
(gdb) target remote IP:PORT                  # 'remote' does not permit resetting

In [ ]:
(gdb) target extended-remote IP:PORT         # 'extended-remote' permits resetting

To load the executable file into the target,

In [ ]:
(gdb) load

To run (or, reset) the loaded executable file,

In [ ]:
(gdb) run

To end the GDB debugging session,

In [ ]:
(gdb) quit

#### Reading from memory

The *Text User Interface (TUI)* mode enables the viewing of source, and/or assembly, as execution proceeds.

In [ ]:
(gdb) tui enable

In [ ]:
(gdb) tui disable

With TUI mode enabled, the `layout` command modifies what is to be displayed.

In [ ]:
(gdb) layout LAYOUT                     # 'src', 'asm', or 'split'

*Note:* If the `file` command has been used to load a different file, GDB does not reload the source/assembly files automatically, relying on cache instead. The `directory` command can be used to reset the file search path to be the compilation directory, if defined in the executable file, and the current working directory. Any alteration in the file search path forces GDB to reload the files.

To display the function calls, leading up to the current instruction,

In [ ]:
(gdb) info stack

To display the current values in the register file,

In [ ]:
(gdb) info registers

To display the current value of a variable, in the current context,

In [ ]:
(gdb) print VARIABLE

In [ ]:
(gdb) print *VARIABLE

To display the current values of all local variables,

In [ ]:
(gdb) info locals

To display the current values of all passed arguments,

In [ ]:
(gdb) info args

To display the current value at a specific address,

In [ ]:
(gdb) info ADDRESS                                   # e.g: '0x40020000'

In [ ]:
(gdb) info ADDRESS + OFFSET                          # e.g: '0x40020000 + 0x14'

#### Writing to memory

To write to an arbitrary memory address,

In [ ]:
(gdb) set *ADDRESS = VALUE                           # Writes a word, by default           

To write to a specific variable, in the current context,

In [ ]:
(gdb) set variable VARIABLE = VALUE                  # 'var' shorthand of 'variable'

In [ ]:
(gdb) set variable *VARIABLE = VALUE

#### Managing breakpoints

A *breakpoint* is an address of an instruction, that when executed, the CPU halts.

* A *hardware breakpoint* is implemented by placing the address of the instruction in special registers within the CPU, called *hardware breakpoint registers*. Continously, the CPU compares the *program counter (PC)* register with the hardware breakpoint registers, and halts if a match occurs.

* A *software breakpoint* is implemented by a debugger, when executing from RAM. Instructions at breakpoints are substituted with a special CPU instruction (e.g: `BKPT` in ARM), that halts the CPU when executed. The debugger, then, ensures the next instruction to be executed is the previously substituted instruction.

By default, in read-only addresses, GDB inserts *hardware breakpoints*, which are limited by the number of supporting registers within the CPU.

To insert a breakpoint,

In [ ]:
(gdb) break FUNCTION

In [ ]:
(gdb) break FILE:LINE

*Note:* Similarly, use the `clear` command to delete a breakpoint.

A *conditional breakpoint* behaves like any breakpoint, forcing the CPU to halt when executing from a specific address. However, the debugger automatically proceeds with the execution, if a condition is not satisified.

To apply a condition on a breakpoint,

In [ ]:
(gdb) condition NUMBER BOOL-EXPR                 # e.g: 'x >= 10 && x <= 20'

*Note:* Within the condition, local variables, in the referenced context, as well as global variables may be used.

A *watchpoint* is an address in memory, that the CPU halts after writing to, or reading from.
* A *hardware watchpoint* is implemented, similar to hardware breakpoints, using special CPU registers.
* A *software watchpoint* is implemented by a debugger, by stepping through each instruction and checking whether the specified address was written to, or read from.

By default, GDB exhausts *hardware watchpoint registers*, and complains when they run out.

To watch a variable, or an arbitrary memory location, in the current context,

In [ ]:
(gdb) watch VARIABLE                           # Breaks on write to 'VARIABLE'

In [ ]:
(gdb) watch *ADDRESS                           # Breaks on write to '*ADDRESS'

In [ ]:
(gdb) awatch ...                               # Breaks on read-write

In [ ]:
(gdb) rwatch ...                               # Breaks on read-only

*Note:* GDB inserts software watchpoints when dealing with local variables, and hence, avoid watching local variables.

To disable a breakpoint, a conditional breakpoint, or a watchpoint,

In [ ]:
(gdb) disable NUMBER                   # 'NUMBER' is the number given to any point by GDB

*Note:* Similarly, use the `enable` command to enable a breakpoint, or `delete` to delete.

To show the status of all existing breakpoints, conditional breakpoints, and watchpoints,

In [ ]:
(gdb) info break

#### Controlling execution flow

After the CPU halts, different commands are used to control how execution proceeds thereafter.

To step through lines in the source file,

In [ ]:
(gdb) step                        # Step through a single line

In [ ]:
(gdb) step N                      # Step through 'N' lines

To step through instructions in the assembly file,

In [ ]:
(gdb) stepi                       # Step through a single instruction

In [ ]:
(gdb) stepi N                     # Step through 'N' instructions

To step over a function call (i.e, execute it without entering the function itself) in the source file,

In [ ]:
(gdb) next

To step out of the currently executing function (i.e, execute until the function returns) in the source file,

In [ ]:
(gdb) finish

To run (or, reset) the loaded executable file, halting at the entry function (e.g: `main`) via a temporary breakpoint,

In [ ]:
(gdb) start